## Project: 15 Million Chess Games from Lichess 2013-2014 Analysis and Libraries Comparisons

###Group Members: 

<table>
  <tr>
    <th>Name</th>
    <th>Matric No</th>
  </tr>
  <tr>
    <th>Eddie Wong Chung Pheng </th>
    <th>A20EC0031</th>
  </tr>
  <tr>
    <th>Madihah Binti Che Zabri </th>
    <th>A20EC0074</th>
  </tr>
  <tr>
    <th>Nurarissa Dayana Binti Mohd Sukri</th>
    <th>A20EC0120</th>
  </tr>
  <tr> 
    <th>Vincent Boo Ee Khai</th>
    <th>A20EC0231</th>
  </tr>
</table>
<br></br>

In [ ]:
# Read dataset from google drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/file/d/1QRzLWLPBAbdPBloNgOgZ4OKmHdQQk8GF/view?usp=sharing'
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Parking Violations Issued - Fiscal Year 2014_August 2013_June_2014.csv') 

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv("Parking Violations Issued - Fiscal Year 2014_August 2013_June_2014.csv")
pd.set_option('display.max_columns', None)
df.head(5)

NameError: ignored